# Single-atlas segmentation of cardiac substructures.

This notebook demonstrates single-atlas segmentation of cardiact substructures.

The data used are from the CASSCADE study.

In [ ]:
from pathlib import Path

from skrt import Patient, set_viewer_options
from skrt.registration import get_default_pfiles
from skrt.segmentation import SingleAtlasSegmentation

# Define paths to patient data.
# Each dataset consists of the CT image used in treatment planning, and an associated structure set.
# The structure set for each target dataset includes contours for the heart.
# The structure set for each atlas dataset includes contours for multiple cardiac substructures.
data_dir = Path("~/data").expanduser()
# Paths to target datasets.
paths1 = sorted(list((data_dir / "casscade").glob('casscade*/[!.]*')))
# Paths to atlas datasets.
paths2 = sorted(list((data_dir / "casscade2").glob("*Atlas_1*")))

# Set paths to directories containing registration software.
engine_dirs = {
    "elastix": "~/sw/elastix-5.0.1-mac",
    "niftyreg": "~/sw/niftyreg",
}

# Make plots interactive ("no_ui": False) or non-interactive ("no_ui": True).
no_ui = False

# Set Matplotlib runtime configuration, and obtain dictionary of BetterViewer options.
options = set_viewer_options(to_exclude="figsize", usetex=True, no_ui=no_ui)

In [ ]:
def load_data(paths, idx=0, to_keep=None, to_remove=None, names=None):
    """
    Return objects for patient, patient's CT image, and associated structure set.
    
    **Parameters:**
    
    paths: list
        List of paths to patient datasets.
        
    idx : int, default=0
        Index in <paths> list of dataset to be considered.
        
    to_keep : list, default=None
        List of names of ROIs to be kept in patient's structure set.
        
    to_remove : list, default=None
        List of names of ROIs to remove from patient's structure set.
        
    name : dict, default=None
        Dictionary for renaming ROIs: a key is a name to be used; the associated
        value is a list of names that may have been assigned.
    """
    # Obtain references to patient, study, CT image and structure set. 
    p = Patient(paths[idx], unsorted_dicom=True)
    s = p.studies[0]
    ct = s.ct_images[0]
    ss = s.ct_structure_sets[0]
    
    # Filter structure set, renaming ROIs as needed.
    ss.filter_rois(to_keep=to_keep, to_remove=to_remove)
    ss.rename_rois(names=names)
    
    return (p, ct, ss)

# ROIs to remove from atlas structure sets.
to_remove = ["*ptv*", "*skin*", "*lung*", "d50"]
# ROIs to keep in target structure sets.
to_keep = ["heart"]
# Ensure that ROI name is always capitalised. 
names = {"Heart" : "heart"}

In [ ]:
# Load target data.
idx1 = 0
p1, ct1, ss1 = load_data(paths1, idx1, to_keep=to_keep, names=names)

In [ ]:
# Load atlas data.
idx2 = 0
p2, ct2, ss2 = load_data(paths2, idx2, to_remove=to_remove, names=names)

In [ ]:
# Define and run single-atlas segmentation.

# ROI for initial alignment, and to define cropping.
roi_to_align = "Heart"
# Margin (mm) to be placed around ROI to befine crop region.
crop_margins = 30
# Define segmentation strategy ("pull" or "push" for elastix, "pull" only for niftyreg).
strategy = "pull"
# Define vosel size.
voxel_size = ct2.get_voxel_size()

# Define and run segmenation for each registration engine.
sass = {}
for engine, engine_dir in engine_dirs.items():
    sass[engine] = SingleAtlasSegmentation(
        engine=engine,
        engine_dir=engine_dir,
        im1=ct1,
        im2=ct2,
        workdir=Path(f"results/{p1.id}_{p2.id}/{engine}"),
        initial_crop_focus=roi_to_align,
        initial_crop_margins=crop_margins,
        initial_alignment=roi_to_align,
        voxel_size1=voxel_size,
        pfiles1={"bspline": get_default_pfiles("*BSpline05*", engine)[0]},
        default_roi_crop_margins=(20, 20, 20),
        voxel_size2=voxel_size,
        auto=True,
        default_step=0,
        default_strategy=strategy,
        overwrite=True,
        capture_output=True,
        keep_tmp_dir = True,
        log_level="INFO",
    )

In [ ]:
# Obtain structure sets for final segmentations.
rois = {}
for engine, sas in sass.items():
    rois[engine] = sas.get_segmentation().interpolate_points(
        dxy=2, smoothness_per_point=0.3)
    rois[engine].name = f"{engine}_{rois[engine].name}"

In [ ]:
# Display initial images.
sas.get_registration().view_init(overlay_legend=True, **options);

In [ ]:
# Display results of final registrations.
for engine, sas in sass.items():
    titles = [f"{engine}: {title}" for title in ["fixed", "transformed moving"]]
    sas.get_registration().view_result(overlay_legend=True, title=titles, **options);

In [ ]:
# Display segmentations.
sas.im1.view(images=sas.im1, rois=list(rois.values()), title=list(sass.keys()),
             zoom=4, init_roi="Heart", annotate_slice={"color": 'yellow', "fontsize": 20},
             **options);

In [ ]:
# Display Jacobian determinants.
images = []
jacobians = []
titles = []
for engine, sas in sass.items():
    reg = sas.get_registration()
    step = reg.steps[-1]
    images.append(reg.transformed_images[step])
    jacobians.append(reg.get_jacobian(step))
    titles.append(f"{engine}: {step}")
    
images[0].view(images=images[1:], jacobian=jacobians, title=titles, colorbar=-1, figsize=(21, 6),
               annotate_slice={"color": 'yellow', "fontsize": 20}, **options);

In [ ]:
# Display atlas segmentation alongside target segmentation.
ct2.view(images=ct1, rois=[ss2, rois["niftyreg"]], title=["manual segmentation", "single-atlas segmentation"],
             zoom=4, init_roi="Heart", annotate_slice={"color": 'yellow', "fontsize": 20},
             **options);